In [1]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import pandas as pd
import random

# Load the particle data
df = pd.read_csv("D:/CUDA Projects/Real-time-Particle-Simulation-with-CUDA/particles.csv")
num_frames = df['frame'].max() + 1
num_particles = df['particle_id'].nunique()

# Get grouped frame data
frame_data = [df[df['frame'] == i] for i in range(num_frames)]

# Initialize PyGame and OpenGL
def init_display():
    pygame.init()
    display = (800, 600)
    pygame.display.set_mode(display, DOUBLEBUF | OPENGL)
    glEnable(GL_DEPTH_TEST)
    gluPerspective(45, display[0] / display[1], 0.1, 100.0)
    glTranslatef(-5, -5, -25)  # Move camera back and down a bit

# Generate random colors for each particle
def generate_random_colors(particle_ids):
    random.seed(42)
    return {
        pid: [random.uniform(0.0, 1.0), random.uniform(0.0, 1.0), random.uniform(0.0, 1.0)]
        for pid in particle_ids
    }

# Draw particles for a given frame
def draw_particles(particles, colors):
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glPointSize(3)
    glBegin(GL_POINTS)
    for _, p in particles.iterrows():
        pid = int(p['particle_id'])
        color = colors[pid]
        glColor3f(color[0], color[1], color[2])  # Set color for each particle
        glVertex3f(p['x'], p['y'], p['z'])
    glEnd()
    pygame.display.flip()

# Run animation loop
def run_simulation():
    init_display()
    clock = pygame.time.Clock()
    frame_idx = 0

    # Generate random colors for each particle
    colors = generate_random_colors(df['particle_id'].unique())

    running = True
    while running:
        clock.tick(30)  # Target 30 FPS
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Draw particles for the current frame
        draw_particles(frame_data[frame_idx], colors)

        frame_idx = (frame_idx + 1) % num_frames

    pygame.quit()

if __name__ == "__main__":
    run_simulation()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
